<a href="https://colab.research.google.com/github/josocjo/Colab-akta-chromatogram-viewer/blob/master/interactive_chromatogram_viewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <b>💻 Run interactive application</b> { display-mode: "form" }

!git clone https://github.com/josocjo/Colab-akta-chromatogram-viewer.git
%cd Colab-akta-chromatogram-viewer
!pip install .
from IPython.display import clear_output, display
clear_output()


import os
import io
import contextlib
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
import proteovis as pv
from IPython.display import display, clear_output

# ─────────────────────────────────────────────────────────────
# FUNCIONS AUXILIARS
# ─────────────────────────────────────────────────────────────

def slider_text_pair(description, init, minv, maxv, step=1.0):
    slider = widgets.FloatSlider(value=init, min=minv, max=maxv, step=step,
                                 description=description,
                                 layout=widgets.Layout(width="400px"),
                                 style={'description_width': '150px'})
    text = widgets.FloatText(value=init, layout=widgets.Layout(width="100px"))

    def on_text_change(change):
        val = change['new']
        if val < slider.min: slider.min = val
        if val > slider.max: slider.max = val
        slider.value = val

    def on_slider_change(change):
        text.value = change['new']

    text.observe(on_text_change, names='value')
    slider.observe(on_slider_change, names='value')

    return widgets.HBox([slider, text]), slider

def carregar_fitxer(upload_widget):
    file_name = list(upload_widget.value.values())[0]['metadata']['name']
    file_bytes = list(upload_widget.value.values())[0]['content']
    with open("akta_input", "wb") as f:
        f.write(file_bytes)

    ext = os.path.splitext(file_name)[1].lower()
    with contextlib.redirect_stdout(io.StringIO()), warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)

        if ext in[".zip",".result"]:
            data = pv.pycorn.load_uni_zip("akta_input")
            df = pv.pycorn.utils.get_series_from_data(data, list(data.keys()))
        elif ext in [".res"]:
            res_obj = pv.pycorn.pc_res3("akta_input")
            res_obj.load()
            data = res_obj
            df = pd.DataFrame()
        else:
            raise ValueError(f"Unsupported file extension: {ext}")

    for name, entry in data.items():
        if isinstance(entry, dict) and entry.get("data_type") == "curve":
            try:
                x, y = zip(*entry["data"])
                if "mL" not in df:
                    df["mL"] = x
                df[name] = y
            except:
                pass

    df["Fractions"] = pd.Series(dtype=object)

    try:
        x, labels = zip(*data["Fractions"]["data"])
        for frac_ml, label in zip(x, labels):
            idx = (df["mL"] - frac_ml).abs().idxmin()
            df.at[idx, "Fractions"] = label
    except: pass

    if "UV 2_260" in df.columns and "UV 1_280" in df.columns:
        df["260/280"] = df["UV 2_260"] / df["UV 1_280"]
        data["260/280"] = {"unit": "ratio"}

    return df, data, file_name

def get_senyals(df):
    keys = list(df.columns)
    possibles_uv = [k for k in keys if "UV" in k.upper() and df[k].dtype != "O"]
    possibles_y2 = [k for k in keys if k not in possibles_uv and df[k].dtype != "O" and k not in ["mL", "Fractions"]]
    return possibles_uv, possibles_y2

def plot_chromatogram(df, data, y1a_label, y1b_label, xmin, xmax, ymin, ymax,
                      y2_label, y2_ymin, y2_ymax,
                      show_fractions, tick_h, min_spacing, font_frac,
                      font_title, font_labels, font_ticks, font_legend,
                      file_name):
    fig, ax1 = plt.subplots(figsize=(16, 6))
    if y1a_label in df.columns:
        ax1.plot(df["mL"], df[y1a_label], label=y1a_label, color="blue")
    if y1b_label in df.columns and y1b_label != y1a_label:
        ax1.plot(df["mL"], df[y1b_label], label=y1b_label, color="red")
    ax1.set_xlim(xmin, xmax)
    ax1.set_ylim(ymin, ymax)
    ax1.set_xlabel("Elution volume (mL)", fontsize=font_labels)
    ax1.set_ylabel("Absorbance (mAU)", fontsize=font_labels)
    ax1.tick_params(axis='both', labelsize=font_ticks)
    ax1.grid(True)

    if show_fractions and "Fractions" in df.columns:
        last_label_x = None
        fractions = df[(df['Fractions'].notna()) & (df['mL'].between(xmin, xmax))].reset_index()
        for i in range(len(fractions)):
            x = fractions.loc[i, 'mL']
            ax1.vlines(x, ymin, ymin + tick_h, color='red', linewidth=1)
            if i % 2 == 0 and (last_label_x is None or abs(x - last_label_x) > min_spacing):
                label = fractions.loc[i, 'Fractions']
                try:
                    txt = 'W' if str(label).lower() == 'waste' else str(int(label))
                except:
                    txt = str(label)
                ax1.text(x, ymin + tick_h + 5, txt, color='black', fontsize=font_frac,
                         ha='center', va='bottom', rotation=0, clip_on=False)
                last_label_x = x

    ax2 = None
    if y2_label and y2_label in df.columns:
        unit = data.get(y2_label, {}).get("unit", "")
        label_with_unit = f"{y2_label} ({unit})" if unit else y2_label
        ax2 = ax1.twinx()
        ax2.plot(df["mL"], df[y2_label], label=label_with_unit, color="green")
        ax2.set_ylabel(label_with_unit, color="green", fontsize=font_labels)
        ax2.tick_params(axis='y', labelcolor='green', labelsize=font_ticks)
        ax2.set_ylim(y2_ymin, y2_ymax)

    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels() if ax2 else ([], [])
    ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper right', fontsize=font_legend)
    fig.subplots_adjust(bottom=0.12)
    fig.suptitle(f'Chromatogram – {file_name}', fontsize=font_title)
    plt.show()

def visualitzar(df, data, file_name):
    possibles_uv, possibles_y2 = get_senyals(df)

    y1a = "UV 1_280" if "UV 1_280" in possibles_uv else possibles_uv[0]
    y1b = "UV 2_260" if "UV 2_260" in possibles_uv else possibles_uv[0]

    y1a_dropdown = widgets.Dropdown(options=possibles_uv, value=y1a, description="UV 1")
    y1b_dropdown = widgets.Dropdown(options=possibles_uv, value=y1b, description="UV 2")
    y2_dropdown = widgets.Dropdown(options=[""] + possibles_y2, value=possibles_y2[0] if possibles_y2 else "", description="Second Y")

    uv_max = df[[col for col in [y1a, y1b] if col in df.columns]].max().max()
    ml_max = df["mL"].max()

    xmin_box, xmin = slider_text_pair("xmin", df["mL"].min(), df["mL"].min(), ml_max)
    xmax_box, xmax = slider_text_pair("xmax", ml_max, df["mL"].min(), ml_max)
    ymin_box, ymin = slider_text_pair("ymin", 0, -100, uv_max + 50)
    ymax_box, ymax = slider_text_pair("ymax", uv_max + 50, 10, uv_max + 300)
    y2_ymin_box, y2_ymin = slider_text_pair("Y2 min", 0, -20, 100)
    y2_ymax_box, y2_ymax = slider_text_pair("Y2 max", 100, 10, 300)
    show_fractions = widgets.Checkbox(value=True, description='Show fractions')
    tick_h_box, tick_h = slider_text_pair("Tick height", 10, 1, 50)
    spacing_box, min_spacing = slider_text_pair("Min spacing", 1.0, 0.1, 10.0)
    font_frac_box, font_frac = slider_text_pair("Font frac", 12, 6, 30)
    font_title_box, font_title = slider_text_pair("Font title", 18, 10, 30)
    font_labels_box, font_labels = slider_text_pair("Font labels", 18, 10, 30)
    font_ticks_box, font_ticks = slider_text_pair("Font ticks", 18, 8, 30)
    font_legend_box, font_legend = slider_text_pair("Font legend", 18, 8, 30)

    def on_y2_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            col = change['new']
            if col and col in df.columns:
                y2_ymin.value = float(df[col].min()) - 5
                y2_ymax.value = float(df[col].max()) + 10
    y2_dropdown.observe(on_y2_change)

    controls_top = widgets.HBox([
        widgets.VBox([xmin_box, xmax_box]),
        widgets.VBox([ymin_box, ymax_box]),
        widgets.VBox([y2_ymin_box, y2_ymax_box])
    ])
    controls_bottom = widgets.HBox([
        widgets.VBox([y2_dropdown, y1a_dropdown, y1b_dropdown]),
        widgets.VBox([show_fractions, tick_h_box, spacing_box, font_frac_box]),
        widgets.VBox([font_title_box, font_labels_box, font_ticks_box, font_legend_box])
    ])

    out = widgets.interactive_output(
        lambda **kwargs: plot_chromatogram(df, data, **kwargs, file_name=file_name),
        {
            'y1a_label': y1a_dropdown, 'y1b_label': y1b_dropdown,
            'xmin': xmin, 'xmax': xmax,
            'ymin': ymin, 'ymax': ymax,
            'y2_label': y2_dropdown,
            'y2_ymin': y2_ymin, 'y2_ymax': y2_ymax,
            'show_fractions': show_fractions,
            'tick_h': tick_h,
            'min_spacing': min_spacing,
            'font_frac': font_frac,
            'font_title': font_title,
            'font_labels': font_labels,
            'font_ticks': font_ticks,
            'font_legend': font_legend
        })

    display(widgets.HTML("<hr><h4>Plot parameters:</h4>"))
    display(controls_top)
    display(out)
    display(widgets.HTML("<h4>Adjust axis ranges:</h4>"))
    display(controls_bottom)

    # ─────────────────────────────────────────────────────────────
    # ANÀLISI DE FRACCIONS
    # ─────────────────────────────────────────────────────────────
    display(widgets.HTML("<hr><h4>🔬 Fraction analysis:</h4>"))

    fr_start = widgets.Text(value="", description="Start:")
    fr_end = widgets.Text(value="", description="End:")
    epsilon_box = widgets.FloatText(value=10000, description="ε (M⁻¹·cm⁻¹):")
    pathlength_box = widgets.FloatText(value=1.0, description="Path (cm):")
    pm_box = widgets.FloatText(value=50000, description="PM (Da):")
    run_button = widgets.Button(description="Run analysis", button_style='success')

    controls_row = widgets.HBox([fr_start, fr_end, epsilon_box, pathlength_box, pm_box, run_button])
    output_table = widgets.Output()

    def run_analysis(_=None):
        with output_table:
            clear_output()
            try:
                # Obtenir el rang de start i end, verificant si són números o strings
                try:
                    start_label = int(fr_start.value)  # provar convertir a enter
                    end_label = int(fr_end.value)      # provar convertir a enter
                    start_is_numeric = True  # És numèric
                except ValueError:
                    start_label = fr_start.value  # Es conserva com string
                    end_label = fr_end.value      # Es conserva com string
                    start_is_numeric = False  # No és numèric

                # Filtrar les fraccions vàlides (es fa còpia explícita)
                df_valid = df[df["Fractions"].notna()].copy()  # Aquesta línia ja fa còpia
                if "Fractions" not in df_valid.columns or df_valid.empty:
                    print("No valid fraction data found.")
                    return

                if start_is_numeric:  # Si els valors són numèrics
                    # Filtrar per fraccions numèriques
                    selected = df_valid[df_valid["Fractions"].apply(lambda x: x.split(".")[-1].isdigit())].copy()  # Afegim .copy() aquí
                    selected["Numeric Fractions"] = selected["Fractions"].apply(lambda x: int(x.split(".")[-1]))
                    selected = selected[selected["Numeric Fractions"].between(start_label, end_label)]
                else:  # Si són strings
                    # Convertir les fraccions originals a mL per comparar-les
                    start_ml = df_valid[df_valid["Fractions"] == start_label]["mL"].min()
                    end_ml = df_valid[df_valid["Fractions"] == end_label]["mL"].max()

                    if pd.isna(start_ml) or pd.isna(end_ml):
                        print(f"Invalid fraction range: {start_label} to {end_label}")
                        return

                    # Filtrar les fraccions per mL (ordenades per volum d'elució)
                    selected = df_valid[df_valid["mL"].between(start_ml, end_ml)].copy()  # Afegim .copy() aquí

                # Ordenar les fraccions per mL
                selected = selected.sort_values("mL")

                if selected.empty:
                    print(f"No fractions found between {start_label} and {end_label}.")
                    return

                # Detectar UV disponible per fer el càlcul
                candidate_uvs = [col for col in selected.columns if "UV" in col]
                if not candidate_uvs:
                    print("No UV data available to compute concentration.")
                    return

                uv_used = candidate_uvs[0]  # Escollim el primer UV disponible

                # Preparar el resultat
                cols_to_show = ["Fractions", "mL", uv_used]
                if "260/280" in selected.columns:
                    cols_to_show.append("260/280")

                result = selected[cols_to_show].copy()  # Afegim .copy() aquí també

                # Càlcul de concentració
                result["Conc (mol/L)"] = (result[uv_used] / 1000) / (epsilon_box.value * pathlength_box.value)
                result["Conc (mg/mL)"] = result["Conc (mol/L)"] * pm_box.value

                # Afegir la fracció original a la taula de resultats
                result["Original Fraction"] = result["Fractions"].apply(lambda x: df.loc[df["Fractions"] == x, "Fractions"].values[0] if x in df["Fractions"].values else "")

                display(widgets.HTML(f"<b>Concentration calculated using signal: <code>{uv_used}</code></b>"))
                display(result.reset_index(drop=True))

            except Exception as e:
                print(f"Error during analysis: {e}")







    run_button.on_click(run_analysis)

    display(controls_row)
    display(output_table)

# ─────────────────────────────────────────────────────────────
# EXECUCIÓ
# ─────────────────────────────────────────────────────────────

# ─────────────────────────────────────────────────────────────
# EXECUCIÓ ROBUSTA – PERMET CARREGAR MÚLTIPLES FITXERS
# ─────────────────────────────────────────────────────────────

output_zone = widgets.Output()

import time

def crear_upload():
    upload_widget = widgets.FileUpload(accept='.zip,.res,.result', multiple=False)

    def carregar_i_visualitzar(change):
        if len(upload_widget.value) == 0:
            with output_zone:
                clear_output()
                print("No file uploaded.")
            return
        try:
            # Netejar l'entorn abans de carregar un nou fitxer
            clear_output(wait=True)
            df, data, file_name = carregar_fitxer(upload_widget)
            visualitzar(df, data, file_name)
            display(crear_upload())  # Mostrar el nou widget aquí
        except Exception as e:
            with output_zone:
                clear_output()
                print(f"Error loading the file: {e}")
                time.sleep(0.1)
                display(crear_upload())

    upload_widget.observe(carregar_i_visualitzar, names='value')
    return upload_widget


# Mostra el primer widget de càrrega i la zona de sortida
display(widgets.HTML("<h3>Upload a .zip, .res or .result file:</h3>"))
display(crear_upload())
display(output_zone)
